In [3]:
import pandas as pd
from iTransformer import iTransformer

# Load your CSV
data = pd.read_csv('flattened_dataset.csv')

# Drop the heading (if needed) to get the shape (50000, 832)
data = data.iloc[1:]  # Adjust if headings are in the first row


In [4]:
data.shape

(49999, 833)

In [2]:
import numpy as np
import torch

sequence_length = 100
num_features = 832
num_variates = num_features



sequences = []
targets = []

for i in range(len(data) - sequence_length):
    sequences.append(data.iloc[i:i+sequence_length, 1:].values)
    targets.append(data.iloc[i+sequence_length, 1:].values)

sequences = np.array(sequences)
targets = np.array(targets)

# Convert to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.float32)
targets = torch.tensor(targets, dtype=torch.float32)

# Split data into training and validation sets
train_size = int(0.8 * len(sequences))
train_sequences = sequences[:train_size]
train_targets = targets[:train_size]
val_sequences = sequences[train_size:]
val_targets = targets[train_size:]

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
model = iTransformer(
    num_variates=num_variates,
    lookback_len=sequence_length,
    depth=6,  # Adjust depth as needed
    dim=128,  # Adjust dimension as needed
    pred_length=1  # Predicting the next timestep
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
model.train()
epochs = 10  # Number of epochs
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = model(train_sequences, train_targets)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


In [ ]:
model.fit(train_sequences, train_targets, val_data=(val_sequences, val_targets))


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
model.eval()
with torch.no_grad():
    predictions = model(val_sequences)

predictions = predictions.numpy()
val_targets = val_targets.numpy()

mse = mean_squared_error(val_targets, predictions)
mae = mean_absolute_error(val_targets, predictions)
r2 = r2_score(val_targets, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R² Score: {r2}")
